In [1]:
import copy
import datetime as dt
import importlib # needed so that we can reload packages
import matplotlib.pyplot as plt
import os, os.path
import numpy as np
import pandas as pd
import pathlib
import sys
import time
from typing import Union
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")

##  IMPORT SISEPUEDE EXAMPLES AND TRANSFORMERS

from sisepuede.manager.sisepuede_examples import SISEPUEDEExamples
from sisepuede.manager.sisepuede_file_structure import SISEPUEDEFileStructure
import sisepuede.core.support_classes as sc
import sisepuede.transformers as trf
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf

In [2]:
%load_ext autoreload
%autoreload 2

### Initial Set up

Make sure to edit the config yaml under croatia/config_files/croatia_config.yaml

You can also create a new config yaml



In [3]:
# Set up dir paths

dir_cur = pathlib.Path(os.getcwd())
dir_ssp_modeling = dir_cur.parent
dir_project = dir_ssp_modeling.parent
dir_data = dir_ssp_modeling.joinpath("input_data")
dir_scenario_mapping = dir_ssp_modeling.joinpath("scenario_mapping")
dir_config = dir_ssp_modeling.joinpath("config_files")
dir_transformations = dir_ssp_modeling.joinpath("transformations")
dir_misc = dir_ssp_modeling.joinpath("misc")
strategies_definitions_file_path = dir_transformations.joinpath("strategy_definitions.csv")
strategy_mapping_file_path = dir_misc.joinpath("strategy_mapping.yaml")

In [4]:
path_ok = os.path.join(dir_project.parent, 'src/ssp_transformations_handler')

In [5]:
import sys
sys.path.append(path_ok)
from GeneralUtils import GeneralUtils
from TransformationUtils import TransformationYamlProcessor, StrategyCSVHandler

# Initialize general utilities
g_utils = GeneralUtils()

In [6]:
# Load config file, double check your parameters are correct

yaml_file_path = os.path.join(dir_config, "mexico_config.yaml")
config_params = g_utils.read_yaml(yaml_file_path)
country_name = config_params['country_name']
ssp_input_file_name = config_params['ssp_input_file_name']
ssp_transformation_cw = config_params['ssp_transformation_cw']
energy_model_flag = config_params['energy_model_flag']


In [7]:
# Set up SSP objects

fp_data_base = dir_data.joinpath(ssp_input_file_name)

file_struct = SISEPUEDEFileStructure()

matt = file_struct.model_attributes
regions = sc.Regions(matt)
time_periods = sc.TimePeriods(matt)

### Making sure our input file has the correct format and correct columns
We use an example df with the complete fields and correct format to make sure our file is in the right shape

In [8]:
##  BUILD BASE INPUTS

df_inputs_raw = pd.read_csv(fp_data_base)

# pull example data to fill in gaps
examples = SISEPUEDEExamples()
df_inputs_example = examples.input_data_frame

In [9]:
g_utils.check_frac_groups(df_inputs_raw, os.path.join(dir_misc, "energy_frac_vars.xlsx"))

Row sums for frac_enfu_fuel_demand are not in range the min is 1.9577434850274216 and the max is 5.119161135061601. Please check the data.
Row sums for frac_trns_fuelmix_powered are not in range the min is 0.9999999999999999 and the max is 1.0000000000000002. Please check the data.
Row sums for frac_trns_fuelmix_rail_passenger are not in range the min is 1.9999999999999998 and the max is 2.0. Please check the data.


In [10]:
df_inputs_example.head()

,region,time_period,avgload_trns_freight_tonne_per_vehicle_aviation,avgload_trns_freight_tonne_per_vehicle_rail_freight,avgload_trns_freight_tonne_per_vehicle_road_heavy_freight,avgload_trns_freight_tonne_per_vehicle_water_borne,avgmass_lvst_animal_buffalo_kg,avgmass_lvst_animal_cattle_dairy_kg,avgmass_lvst_animal_cattle_nondairy_kg,avgmass_lvst_animal_chickens_kg,...,ef_ippu_tonne_cf4_per_tonne_production_electronics,ef_ippu_tonne_cf4_per_tonne_production_metals,ef_ippu_tonne_c3f8_per_tonne_production_chemicals,ef_ippu_tonne_c3f8_per_tonne_production_electronics,ef_ippu_tonne_c4f10_per_mmm_gdp_product_use_ods_other,ef_ippu_tonne_c4f10_per_tonne_production_chemicals,ef_ippu_tonne_c6f14_per_mmm_gdp_product_use_ods_other,ef_ippu_tonne_c6f14_per_tonne_production_chemicals,ef_ippu_tonne_cc4f8_per_tonne_production_chemicals,ef_ippu_tonne_cc4f8_per_tonne_production_electronics
0,costa_rica,0,70.0,2923.0,31.751466,6468.0,322.900664,520.741388,310.599686,1.12759,...,0.000002,4.204879e-07,0.0,1.515048e-07,0.0,0.0,0.0,0.0,0.0,5.669821e-08
1,costa_rica,1,70.0,2923.0,31.751466,6468.0,322.900664,520.741388,310.599686,1.12759,...,0.000002,2.134675e-07,0.0,1.313925e-07,0.0,0.0,0.0,0.0,0.0,5.185820e-08
2,costa_rica,2,70.0,2923.0,31.751466,6468.0,322.900664,520.741388,310.599686,1.12759,...,0.000002,1.821067e-07,0.0,1.060066e-07,0.0,0.0,0.0,0.0,0.0,5.269348e-08
3,costa_rica,3,70.0,2923.0,31.751466,6468.0,322.900664,520.741388,310.599686,1.12759,...,0.000002,2.094712e-07,0.0,1.093024e-07,0.0,0.0,0.0,0.0,0.0,5.500450e-08
4,costa_rica,4,70.0,2923.0,31.751466,6468.0,322.900664,520.741388,310.599686,1.12759,...,0.000002,3.446161e-07,0.0,1.046451e-07,0.0,0.0,0.0,0.0,0.0,5.266081e-08


In [11]:
# Checks if there are any differences in columns between the example df and our df
g_utils.compare_dfs(df_inputs_example, df_inputs_raw)


Columns in df_example but not in df_input: {'frac_trns_fuelmix_water_borne_ammonia', 'cost_enfu_fuel_ammonia_usd_per_tonne', 'nemomod_entc_emissions_activity_ratio_scalar_fp_hydrogen_reformation_ccs_co2', 'nemomod_entc_fixed_cost_fp_ammonia_production_mm_usd_per_gw', 'strategy_id', 'frac_enfu_fuel_demand_imported_pj_fuel_ammonia', 'ef_enfu_combustion_tonne_co2_per_tj_fuel_ammonia', 'nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_oil', 'nemomod_entc_output_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_hydrogen', 'nemomod_entc_capital_cost_fp_hydrogen_reformation_ccs_mm_usd_per_gw', 'ef_enfu_stationary_combustion_tonne_ch4_per_tj_fuel_ammonia', 'nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_electricity', 'nemomod_entc_input_activity_ratio_fuel_production_fp_ammonia_production_electricity', 'nemomod_entc_output_activity_ratio_fuel_production_fp_ammonia_production_ammonia', 'region', 'energydensity_gravimetric_enfu

In [12]:
help(g_utils.remove_additional_cols)

Help on function remove_additional_cols in module GeneralUtils:

remove_additional_cols(df_example, df_input)
    Remove columns from df_input that are not present in df_example.
    Parameters:
    df_example (pandas.DataFrame): The reference DataFrame containing the desired columns.
    df_input (pandas.DataFrame): The DataFrame from which additional columns will be removed.
    Returns:
    pandas.DataFrame: A DataFrame with only the columns present in df_example.



In [13]:
# Fixes differences and makes sure that our df is in the correct format.
# Note: Edit this if you need more changes in your df

df_inputs_raw = df_inputs_raw.rename(columns={'period':'time_period'})
df_inputs_raw_complete = g_utils.add_missing_cols(df_inputs_example, df_inputs_raw.copy())
df_inputs_raw_complete = g_utils.remove_additional_cols(df_inputs_example, df_inputs_raw_complete.copy())
df_inputs_raw_complete = df_inputs_raw_complete.drop(columns='iso_code3', errors='ignore')
df_inputs_raw_complete.head()

,pij_lndu_forests_primary_to_croplands,pij_lndu_forests_mangroves_to_wetlands,yf_agrc_nuts_tonne_ha,ef_lndu_conv_forests_secondary_to_other_gg_co2_ha,ef_lndu_conv_forests_secondary_to_wetlands_gg_co2_ha,ef_lndu_conv_other_to_forests_primary_gg_co2_ha,pij_lndu_forests_mangroves_to_forests_secondary,ef_lndu_conv_wetlands_to_settlements_gg_co2_ha,pij_lndu_grasslands_to_forests_secondary,frac_frst_primary_cl1_tropical,...,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_electrolysis_water,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_electricity,energydensity_gravimetric_enfu_gj_per_tonne_fuel_ammonia,energydensity_gravimetric_enfu_gj_per_tonne_fuel_water,frac_trns_fuelmix_water_borne_ammonia,nemomod_entc_output_activity_ratio_fuel_production_fp_ammonia_production_ammonia,nemomod_entc_output_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_hydrogen,nemomod_entc_frac_min_share_production_fp_hydrogen_reformation_ccs,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_natural_gas,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_oil
0,0.000000,0.0,1.497014,0.128343,0.129618,0.0,0.0,0.0,0.0,0.249929,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
1,0.000000,0.0,1.438057,0.128343,0.129618,0.0,0.0,0.0,0.0,0.249929,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
2,0.000000,0.0,1.435714,0.128343,0.129618,0.0,0.0,0.0,0.0,0.249929,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
3,0.000016,0.0,1.464529,0.128343,0.129618,0.0,0.0,0.0,0.0,0.249929,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
4,0.000031,0.0,1.452086,0.128343,0.129618,0.0,0.0,0.0,0.0,0.249929,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0


In [14]:
# Double checking that our df is in the correct shape (Empty sets should be printed to make sure everything is Ok!)
g_utils.compare_dfs(df_inputs_example, df_inputs_raw_complete)

Columns in df_example but not in df_input: set()
Columns in df_input but not in df_example: set()


In [15]:
# Checking if there are any columns with null values in it
columns_with_na = df_inputs_raw_complete.columns[df_inputs_raw_complete.isna().any()].tolist()

print(columns_with_na)

[]


In [16]:
# Checking region to avoid copy errors
df_inputs_raw_complete['region'].head()

0    costa_rica
1    costa_rica
2    costa_rica
3    costa_rica
4    costa_rica
Name: region, dtype: object

In [17]:
# Set region to country name
df_inputs_raw_complete['region'] = country_name
df_inputs_raw_complete['region'].head()

0    mexico
1    mexico
2    mexico
3    mexico
4    mexico
Name: region, dtype: object

In [18]:
# Normalize ENERGY frac_vars
g_utils.check_frac_groups(df_inputs_raw_complete, os.path.join(dir_misc, "energy_frac_vars.xlsx"))

Row sums for frac_enfu_fuel_demand are not in range the min is 1.9577434850274216 and the max is 5.119161135061601. Please check the data.
Row sums for frac_trns_fuelmix_powered are not in range the min is 0.9999999999999999 and the max is 1.0000000000000002. Please check the data.
Row sums for frac_trns_fuelmix_rail_passenger are not in range the min is 1.5086206896551724 and the max is 1.5222929936305731. Please check the data.


In [19]:
df_inputs_raw_complete[[col for col in df_inputs_raw_complete.columns if 'frac_enfu_fuel_demand' in col]]

,frac_enfu_fuel_demand_imported_pj_fuel_kerosene,frac_enfu_fuel_demand_imported_pj_fuel_oil,frac_enfu_fuel_demand_imported_pj_fuel_electricity,frac_enfu_fuel_demand_imported_pj_fuel_hydrocarbon_gas_liquids,frac_enfu_fuel_demand_imported_pj_fuel_natural_gas,frac_enfu_fuel_demand_imported_pj_fuel_coal,frac_enfu_fuel_demand_imported_pj_fuel_crude,frac_enfu_fuel_demand_imported_pj_fuel_gasoline,frac_enfu_fuel_demand_imported_pj_fuel_diesel,frac_enfu_fuel_demand_imported_pj_fuel_hydrogen,frac_enfu_fuel_demand_imported_pj_fuel_ammonia
0,0.296034,0.002069,0.006312,0.296034,0.384384,0.353773,0.002069,0.296034,0.296034,0.025,0
1,0.336056,0.002677,0.008129,0.336056,0.442948,0.390111,0.002677,0.336056,0.336056,0.025,0
2,0.857224,0.003961,0.543897,0.857224,0.615376,0.498072,0.003961,0.857224,0.857224,0.025,0
3,0.336056,0.002677,0.008129,0.336056,0.442948,0.390111,0.002677,0.336056,0.336056,0.025,0
4,0.336056,0.002677,0.008129,0.336056,0.442948,0.390111,0.002677,0.336056,0.336056,0.025,0
5,0.336056,0.002677,0.008129,0.336056,0.442948,0.390111,0.002677,0.336056,0.336056,0.025,0
6,0.336056,0.002677,0.008129,0.336056,0.442948,0.390111,0.002677,0.336056,0.336056,0.025,0
7,0.336056,0.002677,0.008129,0.336056,0.442948,0.390111,0.002677,0.336056,0.336056,0.025,0
8,0.336056,0.002677,0.008129,0.336056,0.442948,0.390111,0.002677,0.336056,0.336056,0.025,0
9,0.336056,0.002677,0.008129,0.336056,0.442948,0.390111,0.002677,0.336056,0.336056,0.025,0


In [20]:
g_utils.check_individual_frac_vars(df_inputs_raw_complete)

Values in column frac_entc_max_elec_production_increase_to_satisfy_msp_pp_geothermal are not within the range [0, 1]. Please check the data.
Values in column frac_entc_max_elec_production_increase_to_satisfy_msp_pp_wind are not within the range [0, 1]. Please check the data.
Values in column frac_entc_max_elec_production_increase_to_satisfy_msp_pp_ocean are not within the range [0, 1]. Please check the data.
Values in column frac_entc_max_elec_production_increase_to_satisfy_msp_pp_biomass are not within the range [0, 1]. Please check the data.
Values in column frac_entc_max_elec_production_increase_to_satisfy_msp_pp_nuclear are not within the range [0, 1]. Please check the data.
Values in column frac_entc_max_elec_production_increase_to_satisfy_msp_pp_gas_ccs are not within the range [0, 1]. Please check the data.
Values in column frac_entc_max_elec_production_increase_to_satisfy_msp_pp_biogas are not within the range [0, 1]. Please check the data.
Values in column frac_entc_max_elec_p

In [21]:
g_utils.check_individual_frac_vars(df_inputs_example)

Values in column frac_entc_max_elec_production_increase_to_satisfy_msp_pp_biogas are not within the range [0, 1]. Please check the data.
Values in column frac_entc_max_elec_production_increase_to_satisfy_msp_pp_biomass are not within the range [0, 1]. Please check the data.
Values in column frac_entc_max_elec_production_increase_to_satisfy_msp_pp_coal are not within the range [0, 1]. Please check the data.
Values in column frac_entc_max_elec_production_increase_to_satisfy_msp_pp_coal_ccs are not within the range [0, 1]. Please check the data.
Values in column frac_entc_max_elec_production_increase_to_satisfy_msp_pp_gas are not within the range [0, 1]. Please check the data.
Values in column frac_entc_max_elec_production_increase_to_satisfy_msp_pp_gas_ccs are not within the range [0, 1]. Please check the data.
Values in column frac_entc_max_elec_production_increase_to_satisfy_msp_pp_geothermal are not within the range [0, 1]. Please check the data.
Values in column frac_entc_max_elec_pr

In [22]:
frac_fuel_demand_cols = [col for col in df_inputs_raw_complete.columns if 'frac_enfu_fuel_demand' in col]
frac_trns_fuelmix_rail_passenger = [col for col in df_inputs_raw_complete.columns if 'frac_trns_fuelmix_rail_passenger' in col]

print(frac_fuel_demand_cols)
print(frac_trns_fuelmix_rail_passenger)

['frac_enfu_fuel_demand_imported_pj_fuel_kerosene', 'frac_enfu_fuel_demand_imported_pj_fuel_oil', 'frac_enfu_fuel_demand_imported_pj_fuel_electricity', 'frac_enfu_fuel_demand_imported_pj_fuel_hydrocarbon_gas_liquids', 'frac_enfu_fuel_demand_imported_pj_fuel_natural_gas', 'frac_enfu_fuel_demand_imported_pj_fuel_coal', 'frac_enfu_fuel_demand_imported_pj_fuel_crude', 'frac_enfu_fuel_demand_imported_pj_fuel_gasoline', 'frac_enfu_fuel_demand_imported_pj_fuel_diesel', 'frac_enfu_fuel_demand_imported_pj_fuel_hydrogen', 'frac_enfu_fuel_demand_imported_pj_fuel_ammonia']
['frac_trns_fuelmix_rail_passenger_hydrogen', 'frac_trns_fuelmix_rail_passenger_diesel', 'frac_trns_fuelmix_rail_passenger_natural_gas', 'frac_trns_fuelmix_rail_passenger_electricity']


In [23]:
# Update the fuel demand columns
df_inputs_raw_complete.update(df_inputs_example[frac_fuel_demand_cols].fillna(0))
df_inputs_raw_complete.update(df_inputs_example[frac_trns_fuelmix_rail_passenger].fillna(0))

In [24]:
df_inputs_raw_complete.to_csv(dir_data.joinpath('mexico_input_fixed.csv'), index=False)

#  Let's try building transformations using this


In [25]:
transformers = trf.transformers.Transformers(
    {},
    df_input = df_inputs_raw_complete,
)

##  Instantiate some transformations. Make sure to run this cell to create the transformations folder for the first time or if you wish to overwrite

In [26]:
# set an ouput path and instantiate
if not dir_transformations.exists():
    trf.instantiate_default_strategy_directory(
        transformers,
        dir_transformations,
    )
else:
    print(f"Directory {dir_transformations} already exists. Skipping instantiation.")


##  --HERE, CUSTOMIZE YOUR TRANSFORMATIONS AND STRATEGIES--

### Customizing transformations and strategies files using TransformationUtils.py classes

In [27]:
# Generate new transformation files based on the excel mapping file. 
# Make sure to have the most updated format for the excel file, check the one used in this notebook for reference.

cw_file_path = os.path.join(dir_scenario_mapping, ssp_transformation_cw)
print(cw_file_path)
excel_yaml_handler = TransformationYamlProcessor(cw_file_path, dir_transformations)

/home/tony-ubuntu/decision_sciences/ssp_transformations_handler/tests/ssp_modeling/scenario_mapping/ssp_mexico_transformation_dummy.xlsx


In [28]:
# This creates transformation yaml files for each strategy in the excel file

# Use if its the first time you are running this
# excel_yaml_handler.process_yaml_files()

# Use if you have already created the yaml files and just want to update them so the transformations with multiple params are not overwritten
# NOTE: This is a temporary solution, we need to find a better way to handle this
excel_yaml_handler.process_yaml_files()

Created new YAML file transformation_agrc_inc_conservation_agriculture.yaml for strategy strategy_M8_EC and set to default because its a special case
Created new YAML file transformation_agrc_inc_conservation_agriculture.yaml for strategy strategy_SV and set to default because its a special case
Created new YAML file transformation_inen_shift_fuel_heat.yaml for strategy strategy_M8_EC and set to default because its a special case
Created new YAML file transformation_lsmm_inc_management_cattle_pigs.yaml for strategy strategy_M8_EC and set to default because its a special case
Created new YAML file transformation_lsmm_inc_management_other.yaml for strategy strategy_M8_EC and set to default because its a special case
Created new YAML file transformation_lsmm_inc_management_poultry.yaml for strategy strategy_M8_EC and set to default because its a special case
Created new YAML file transformation_lvst_dec_enteric_fermentation.yaml for strategy strategy_M8_EC and set to default because its a

In [29]:
# Load the transformations per strategy dictionary so we can pass it to the strategy handler
# You can also check here if the transformations in each strategy are correct
transformation_per_strategy_dict = excel_yaml_handler.get_transformations_per_strategy_dict()
transformation_per_strategy_dict

{'strategy_M8_EC': ['TX:AGRC:DEC_CH4_RICE_STRATEGY_M8_EC',
  'TX:AGRC:DEC_EXPORTS_STRATEGY_M8_EC',
  'TX:AGRC:DEC_LOSSES_SUPPLY_CHAIN_STRATEGY_M8_EC',
  'TX:AGRC:INC_CONSERVATION_AGRICULTURE_STRATEGY_M8_EC',
  'TX:AGRC:INC_PRODUCTIVITY_STRATEGY_M8_EC',
  'TX:CCSQ:INC_CAPTURE_STRATEGY_M8_EC',
  'TX:ENTC:DEC_LOSSES_STRATEGY_M8_EC',
  'TX:ENTC:TARGET_CLEAN_HYDROGEN_STRATEGY_M8_EC',
  'TX:ENTC:TARGET_RENEWABLE_ELEC_STRATEGY_M8_EC',
  'TX:FGTV:DEC_LEAKS_STRATEGY_M8_EC',
  'TX:FGTV:INC_FLARE_STRATEGY_M8_EC',
  'TX:INEN:INC_EFFICIENCY_ENERGY_STRATEGY_M8_EC',
  'TX:INEN:INC_EFFICIENCY_PRODUCTION_STRATEGY_M8_EC',
  'TX:INEN:SHIFT_FUEL_HEAT_STRATEGY_M8_EC',
  'TX:IPPU:DEC_CLINKER_STRATEGY_M8_EC',
  'TX:IPPU:DEC_DEMAND_STRATEGY_M8_EC',
  'TX:IPPU:DEC_HFCS_STRATEGY_M8_EC',
  'TX:IPPU:DEC_N2O_STRATEGY_M8_EC',
  'TX:IPPU:DEC_OTHER_FCS_STRATEGY_M8_EC',
  'TX:IPPU:DEC_PFCS_STRATEGY_M8_EC',
  'TX:LNDU:DEC_DEFORESTATION_STRATEGY_M8_EC',
  'TX:LNDU:DEC_SOC_LOSS_PASTURES_STRATEGY_M8_EC',
  'TX:LNDU:INC_RE

### Creating new strategies
- You can create new strategies from scratch.
- You can also update existing ones.

In [30]:
# Creating new strategies by updating the strategy_definitions file.
# You can edit this to add yours, as many as you want.
csv_handler = StrategyCSVHandler(strategies_definitions_file_path, dir_transformations, strategy_mapping_file_path, transformation_per_strategy_dict)

# Uncomment for completly new strategies version
csv_handler.add_strategy(strategy_group='PFLO', description='Mexico M8 Economia Circular', yaml_file_suffix='M8_EC')
csv_handler.add_strategy(strategy_group='PFLO', description='Mexico M7 Eficiencia Energetica', yaml_file_suffix='M7_EE')
csv_handler.add_strategy(strategy_group='PFLO', description='Mexico M6 Fomento al Transporte Ferroviario', yaml_file_suffix='M6_TF')
csv_handler.add_strategy(strategy_group='PFLO', description='Mexico M5 Estrategia de Trabajo Remoto', yaml_file_suffix='M5_TR')
csv_handler.add_strategy(strategy_group='PFLO', description='Mexico M4 Pacto de Glasgow por la Electromovilidad', yaml_file_suffix='M4_EV')
csv_handler.add_strategy(strategy_group='PFLO', description='Mexico M3 Estrategia de Carbono Azul', yaml_file_suffix='M3_CA')
csv_handler.add_strategy(strategy_group='PFLO', description='Mexico M2 Nuevas Areas Naturales Protegidas', yaml_file_suffix='M2_ANP')
csv_handler.add_strategy(strategy_group='PFLO', description='Mexico Programa Sembrando Vida', yaml_file_suffix='SV')

Updated file with new row: {'strategy_id': 6003, 'strategy_code': 'PFLO:M8_EC', 'strategy': 'M8_EC', 'description': 'Mexico M8 Economia Circular', 'transformation_specification': 'TX:IPPU:DEC_DEMAND_STRATEGY_M8_EC|TX:LNDU:INC_REFORESTATION_STRATEGY_M8_EC|TX:LVST:DEC_EXPORTS_STRATEGY_M8_EC|TX:WASO:INC_ENERGY_FROM_BIOGAS_STRATEGY_M8_EC|TX:IPPU:DEC_CLINKER_STRATEGY_M8_EC|TX:TRNS:INC_OCCUPANCY_LIGHT_DUTY_STRATEGY_M8_EC|TX:AGRC:DEC_LOSSES_SUPPLY_CHAIN_STRATEGY_M8_EC|TX:CCSQ:INC_CAPTURE_STRATEGY_M8_EC|TX:FGTV:DEC_LEAKS_STRATEGY_M8_EC|TX:IPPU:DEC_HFCS_STRATEGY_M8_EC|TX:INEN:INC_EFFICIENCY_ENERGY_STRATEGY_M8_EC|TX:TRDE:DEC_DEMAND_STRATEGY_M8_EC|TX:TRNS:SHIFT_MODE_FREIGHT_STRATEGY_M8_EC|TX:WASO:INC_CAPTURE_BIOGAS_STRATEGY_M8_EC|TX:AGRC:INC_CONSERVATION_AGRICULTURE_STRATEGY_M8_EC|TX:SOIL:DEC_LIME_APPLIED_STRATEGY_M8_EC|TX:TRNS:SHIFT_MODE_REGIONAL_STRATEGY_M8_EC|TX:TRNS:INC_EFFICIENCY_ELECTRIC_STRATEGY_M8_EC|TX:IPPU:DEC_N2O_STRATEGY_M8_EC|TX:WALI:INC_TREATMENT_INDUSTRIAL_STRATEGY_M8_EC|TX:INEN:IN

In [31]:
# Set up the strategy codes you wish to run in ssp
strategies_to_run = [0, 6003, 6004, 6005, 6006, 6007, 6008, 6009, 6010]

### We finished adding new transformation files and strategies so lets load them back

In [ ]:
# then, you can load this back in after modifying (play around with it)
transformations = trf.Transformations(
    dir_transformations,
    transformers = transformers,
)
tab = transformations.attribute_transformation.table

In [ ]:
#  build the strategies -- will export to path
t0 = time.time()
strategies = trf.Strategies(
    transformations,
    export_path = "transformations",
    prebuild = True,
)

t_elapse = sf.get_time_elapsed(t0)
print(f"Strategies defined at {strategies.transformations.dir_init} initialized in {t_elapse} seconds")

In [ ]:
strategies.attribute_table

##  Build our templates
- let's use the default variable groupings for LHS

In [ ]:
# Building excel templates, make sure to include the strategies ids in the strategies attribute as well as the baseline (0)
df_vargroups = examples("variable_trajectory_group_specification")

strategies.build_strategies_to_templates(
    df_trajgroup = df_vargroups,
    include_simplex_group_as_trajgroup = True,
    strategies = strategies_to_run,
)

# Finally, load SISEPUEDE so that we can run it

In [ ]:
country_name

In [ ]:
import sisepuede as si
# timestamp_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
ssp = si.SISEPUEDE(
    "calibrated",
    db_type = "csv",
    # id_str = f"sisepuede_run_2024-11-04T09:23:26.721580",
    initialize_as_dummy = not(energy_model_flag), # no connection to Julia is initialized if set to True
    regions = [country_name],
    strategies = strategies,
    try_exogenous_xl_types_in_variable_specification = True,
)

In [ ]:
not(energy_model_flag)

In [ ]:
# This runs the model, make sure you edit key_stretegy with the strategy ids you want to execute include baseline (0)
dict_scens = {
    ssp.key_design: [0],
    ssp.key_future: [0],
    ssp.key_strategy: strategies_to_run,
}

ssp.project_scenarios(
    dict_scens,
    save_inputs = True,
    include_electricity_in_energy = energy_model_flag
)

In [ ]:
# df_run = ssp.generate_scenario_database_from_primary_key(0)
# df_run = df_run.get("iran")
# df_run.to_csv("/Users/usuario/Desktop/iran_baselines_temp.csv", encoding = "UTF-8", index = None, )

In [ ]:
# Read input and output files
df_out = ssp.read_output(None)
df_in = ssp.read_input(None)

## Some testing for Croatia (you can delete this section for a new country)

In [ ]:
df_out.head()

In [ ]:
# Checking we've got an output df
df_out[[col for col in df_out.columns if 'subsector' in col]].head()

In [ ]:
df_out[[col for col in df_out.columns if 'subsector' in col]].tail()

In [ ]:
cement_df = df_out[['primary_id', 'time_period'] + [col for col in df_out.columns if 'prod_ippu_cement_tonne' in col]]
cement_df['year'] = cement_df['time_period'] + 2015
cement_df[cement_df.primary_id == 0]

In [ ]:
mining_df = df_out[['primary_id', 'time_period'] + [col for col in df_out.columns if 'prod_ippu_mining_tonne' in col]]
mining_df['year'] = mining_df['time_period'] + 2015
mining_df[mining_df.primary_id == 0]

In [ ]:
df_inputs_raw_complete['lndu_reallocation_factor'].mean()

In [ ]:
df_in['lndu_reallocation_factor'].mean()

In [ ]:
df_inputs_raw_complete['elasticity_ippu_cement_production_to_gdp'].head()

In [ ]:
df_in[['primary_id', 'time_period', 'elasticity_ippu_cement_production_to_gdp']].head()

In [ ]:
[col for col in df_out.columns if 'lvst' in col]

In [ ]:
df_out[['primary_id', 'time_period'] + [col for col in df_out.columns if 'demand_lvst' in col]].head(36)

In [ ]:
df_out[['primary_id', 'time_period'] + [col for col in df_out.columns if 'pop_lvst' in col]].head(37)

In [ ]:
df_in[ 'elasticity_protein_in_diet_to_gdppc']

In [ ]:
df_in['frac_gnrl_eating_red_meat']

In [ ]:
df_in[df_in.time_period == 5][['time_period'] + [col for col in df_in.columns if 'pop_lvst' in col]].head(6)

In [ ]:
df_in[df_in.primary_id == 0][['frac_waso_non_recycled_incinerated', 'frac_waso_non_recycled_landfilled', 'frac_waso_non_recycled_open_dump']].head()

In [ ]:
df_qty_open_dump = df_out[[col for col in df_out.columns if 'qty_waso_open_dump' in col]]
df_qty_open_dump

In [ ]:
df_qty_open_dump.head(15).sum(axis=1)

In [ ]:
df_out[df_out.isna().any(axis=1)]

In [ ]:
df_in[[col for col in df_in.columns if 'frac_waso_recycled' in col]]

# option to pass `df_out` back to data_modifications
- do this if the outputs are needed to rescale input demand values to match IEA data

In [ ]:
# (
#     df_out
#     .to_csv(
#         dir_data.joinpath("sisepuede_outputs_iran_preiea.csv"),
#         index = None,
#         encoding = "UTF-8",
#     )
# )

In [ ]:

fig, ax = plt.subplots(figsize = (18, 8))
ax.set_xlabel("Time Period")
ax.set_ylabel("MT Emissions CO2e ")

df_plot = df_out[
    df_out[ssp.key_primary].isin([71071])
]


fields = matt.get_all_subsector_emission_total_fields()#[x for x in df_out.columns if (x.startswith("emission_co2e_subsector_total"))]
dict_format = dict(
    (k, {"color": v}) for (k, v) in
    matt.get_subsector_color_map().items()
)

fig, ax = spu.plot_stack(
    df_plot,
    fields,
    dict_formatting = dict_format,
    field_x = "time_period",
    figtuple = (fig, ax),
)


# Export Wide File (Last Mandatory Step)

In [ ]:
df_out = ssp.read_output(None)
df_in = ssp.read_input(None)


all_primaries = sorted(list(df_out[ssp.key_primary].unique()))

# build if unable to simply read the data frame
if df_in is None:
    df_in = []
     
    for region in ssp.regions:
        for primary in all_primaries: 
            df_in_filt = ssp.generate_scenario_database_from_primary_key(primary)
            df_in.append(df_in_filt.get(region))
    
    df_in = pd.concat(df_in, axis = 0).reset_index(drop = True)




df_export = pd.merge(
    df_out,
    df_in,
    how = "left",
)



# check output directory 
dir_pkg = os.path.join(
    ssp.file_struct.dir_out, 
    f"sisepuede_summary_results_run_{ssp.id_fs_safe}"
)
os.makedirs(dir_pkg) if not os.path.exists(dir_pkg) else None


for tab in ["ATTRIBUTE_STRATEGY"]:
    table_df = ssp.database.db.read_table(tab)
    if table_df is not None:
        table_df.to_csv(
            os.path.join(dir_pkg, f"{tab}.csv"),
            index=None,
            encoding="UTF-8"
        )
    else:
        print(f"Warning: Table {tab} returned None.")


df_primary = (
    ssp
    .odpt_primary
    .get_indexing_dataframe(
        sorted(list(df_out[ssp.key_primary].unique()))
    )
)
    
df_primary.to_csv(
    os.path.join(dir_pkg, f"ATTRIBUTE_PRIMARY.csv"),
    index = None,
    encoding = "UTF-8"
)

df_export.to_csv(
    os.path.join(dir_pkg, f"sisepuede_results_{ssp.id_fs_safe}_WIDE_INPUTS_OUTPUTS.csv"),
    index = None,
    encoding = "UTF-8"
)

In [ ]:
# Getting the directory where the outputs are stored
ssp.file_struct.dir_out

## Some Optional Plotting

In [ ]:
# primary_id = 0

# df_prim = df_out[
#     df_out[
#         ssp.key_primary
#     ]
#     .isin([primary_id])
# ]

# df_in_cur = df_in[
#     df_in[
#         ssp.key_primary
#     ]
#     .isin([primary_id])
# ]


# fields_plot = [
#     x for x in df_prim.columns
#     if "investment" in x
#     and df_prim[x].max() > 0
# ]
# (
#     df_prim[fields_plot]
#     .plot(
#         legend = True,
#     )
# )




In [ ]:


# fig, ax = plt.subplots(figsize = (15, 10))
# df_resid = matt.extract_model_variable(
#     df_in_cur,
#     "NemoMod ResidualCapacity"
# )
# (
#     df_resid
#     .plot
#     .area(
#         ax = ax
#     )
# )

In [ ]:
# df_resid

In [ ]:
# [x for x in df_out.columns if "capacity" in x]

In [ ]:
# ssp.model_attributes.configuration.dict_config


In [ ]:
# modvar = matt.get_variable("NemoMod Generation Capacity")
# modvar.get_from_dataframe(df_prim).head()

In [ ]:
# #[x for x in matt.all_variables if "Consumption" in x]
# [x for x in matt.all_variables if "Production" in x]
# modvar = matt.get_variable("NemoMod Production by Technology")


# unit = matt.get_unit("energy")
# (modvar.get_from_dataframe(df_prim).iloc[7].sum()*unit.convert("PJ", "mwh")/1000)/367669

In [ ]:
# df_inputs

In [ ]:
# modvar = matt.get_variable("Total Energy Consumption from Industrial Energy")#Industrial Energy")
# matt.extract_model_variable(
#     df_prim,
#     modvar
# )